# Explaining the logic.py module
*Author: Chirag Vartak*  
*Date: 23rd March 2016*

## An Introduction
  
  Hello reader.  
  In this IPython notebook, I will help you a little so that you will be more comfortable using the [logic.py](https://github.com/aimacode/aima-python/blob/master/logic.py) module. As you might already know, the `logic.py` module implements the algorithms given in Chapter 6 (Logical Agents), Chapter 7 (First-Order Logic) and Chapter 8 (Inference in First-Order Logic) of the book *Artificial Intelligence: A Modern Approach*.  
  
  Before we begin, if you are unsure of how to use the [aima-python](https://github.com/aimacode/aima-python) repository or are not familiar with IPython notebooks you should read the [intro notebook](https://github.com/aimacode/aima-python/blob/master/intro.ipynb) first. Also, if you are more comfortable with Java than you are with Python we also have the [aima-java](https://github.com/aimacode/aima-java) repository.  
  
  I am assuming that you have read at least Chapter 7 (Logical Agents). You really want to do this if you intend to make sense of anything I tell you in this notebook, or any code in the `logic.py` module, for that matter. If you haven't you should go back and read this chapter first, at least till Sec. 7.5. As a side note, be sure to keep the `logic.py` module open and keep referring to it as you read this notebook. The docstrings of most classes and function are well-written and will give you more insight and in some cases, even examples, of how to use that particular class or function.  
  
  To briefly outline how I will proceed in this notebook, I will start by telling you more about the classes `KB` and `ProbKB`, the classes for the Knowledge Bases that we will be using. Next, we will begin with Propositional Logic; only after we are mostly done with it, we will be getting into First-Order Logic. In Propositional Logic, we will have a look at the class `Expr` and the `expr` function, and try to get more comfortable with using them to create and manipulate logical expressions. We will also play a little with other utility functions created to make working with statements easy. Then, we will construct a knowledge base of a specific situation in the Wumpus World. We will next go through the `tt_entails` function and experiment with it a bit. The `pl_resolution` and `pl_fc_entails` functions will come next.  
  
  So let's get started.

## Knowledge Bases: `KB` and `PropKB`  
  
  The class `KB` is just a template class which you have to inherit to create a knowledge base class that you plan to use. This class reminds you to implement all the methods mentioned here and will scream at you if you forget to. It is, what you might call in Java, an abstract class. The class `PropKB` has been derived from the class `KB` and all the methods have been implemented in here. Let's have a look at these classes in somewhat more detail.  
  
  We see that the class `KB` 4 methods, apart from `__init__`. A point to note here: the `ask` method simply calls the `ask_generator` method. Thus, this one has already been implemented and what you'll have to actually implement when you create your own knowledge base class (if you want to, though I doubt you'll ever need to), will be the `ask_generator` function and not the `ask` function itself.  
  
  The class `PropKB` now.  
* `__init__(self, sentence=None)` : The constructor `__init__` creates a single field `clauses` which will be a list of all the sentences of the knowledge base, in their CNF form.  
* `tell(self, sentence)` : When you want to add a sentence to the KB, you use the `tell` method. This method takes a sentence, converts it to its CNF, extracts all the clauses, and adds all these clauses to the `clauses` field. So, you need not worry about `tell`ing only clauses to the knowledge base. You can `tell` the knowledge base a sentence in any form that you wish; converting it to CNF and adding the resulting clauses will be handled by the `tell` method.  
* `ask_generator(self, query)` : 